In [3]:
import io
import os
import multiprocessing
import requests

import pandas as pd
import tqdm
from bs4 import BeautifulSoup
from pypdf import PdfReader

In [15]:
def _fetch_page_content(page_num=1):
  url = f"https://www.b3.com.br/pt_br/regulacao/oficios-e-comunicados/?pagination={page_num}"

  # ToDo - retry on except
  try:
    response = requests.get(url)
  except Exception as e:
    raise e

  return BeautifulSoup(response.text, 'html.parser')

In [16]:
def _get_pdf_content(url):
  
  # ToDo - retry on except
  return requests.get(url)

In [42]:
from time import sleep
from random import randint

In [53]:
def scrape_b3(page_num):
    if page_num % 5 == 0: 
        sleep(randint(0,10))
        
    save_path = "/home/gabriel/Documentos/projects/poli-capstone-project/data/pdf/"
    soup = _fetch_page_content(page_num)
    content_divs = soup.select('li.accordion-navigation')
    
    base_url = "https://www.b3.com.br"
    data = []

    # For each published communication
    for content in content_divs:
        published_date = content.select("div.least-content")[0].text
        published_title = content.select("div.content p.primary-text")[0].text
        published_abstract = content.select("div.content p.resumo-oficio")[0].text
        published_subject = content.select("div.content p.assunto-oficio")[0].text
        communication_link = content.select("div.content ul li a")[0].get("href", None)
        
        url = base_url + communication_link
        
        pdf_response = _get_pdf_content(url)
        pdf_bytes = pdf_response.content

        file_date = published_date.replace("/", "_")
        file_title = published_title.replace("/", "_")
        file_name = f'{save_path}{file_date}|{file_title}.pdf'

        data.append([published_date, published_title, published_abstract, published_subject, url, file_name])

        with open(file_name, 'wb') as f:
            f.write(pdf_bytes)

        return pd.DataFrame(data, columns=["published_date", "published_title", "published_abstract", "published_subject", "url", "file_name"])

In [54]:
num_processes = 4
pagination = [i for i in range(1, 285)] # There are 284 pages as of 2023-10-29

with multiprocessing.Pool(processes=num_processes) as pool:
    results = list(
        tqdm.tqdm(
        pool.imap(scrape_b3, pagination),
        total=len(pagination)
        )
    )

100%|████████████████████████████████████████████████████████████████| 284/284 [04:50<00:00,  1.02s/it]


In [63]:
save_path = "/home/gabriel/Documentos/projects/poli-capstone-project/data/pdf/"
df = pd.concat(results)

df_replaced = df.assign(
    file_name=df["file_name"].str.replace(save_path, "")
)
df_replaced.head()

,published_date,published_title,published_abstract,published_subject,url,file_name
0,26/10/23,174-2023-PRE-Ofício Circular,"Para o presente programa, serão credenciados d...",Processo para Credenciamento no Programa de Fo...,https://www.b3.com.br/data/files/11/52/56/89/C...,26_10_23|174-2023-PRE-Ofício Circular.pdf
0,24/10/23,099-2023-VNC-Comunicado Externo,"A B3 informa que, a partir do dia 30/10/2023, ...",Novos grupos de negociação para operações estr...,https://www.b3.com.br/data/files/AA/D4/F5/61/4...,24_10_23|099-2023-VNC-Comunicado Externo.pdf
0,10/10/23,164-2023-PRE-Ofício Circular,"A B3 informa que, em 16/10/2023, às 15h, será ...",Tratamento das Carteiras de Índices da B3 em v...,https://www.b3.com.br/data/files/58/50/FF/7B/2...,10_10_23|164-2023-PRE-Ofício Circular.pdf
0,05/10/23,051-2023-VPC-Comunicado Externo,"Informamos que, a partir de 15/01/2024, as tab...",Reajuste de Preços dos Serviços de Conectivida...,https://www.b3.com.br/data/files/E4/92/39/AB/6...,05_10_23|051-2023-VPC-Comunicado Externo.pdf
0,28/09/23,090-2023-VNC-Comunicado Externo,A B3 informa que está disponível para download...,Gateway Binário – Atualização do Template,https://www.b3.com.br/data/files/3B/61/7B/62/9...,28_09_23|090-2023-VNC-Comunicado Externo.pdf


In [65]:
df_replaced.to_parquet("../data/list_b3.parquet", engine="pyarrow")

In [ ]:
def scrape_b3(page_num):
  soup = _fetch_page_content(page_num)
  content_divs = soup.select('li.accordion-navigation')

  base_url = "https://www.b3.com.br"
  data = []

  # For each published communication
  for content in content_divs:
    published_date = content.select("div.least-content")[0].text
    published_title = content.select("div.content p.primary-text")[0].text
    published_abstract = content.select("div.content p.resumo-oficio")[0].text
    published_subject = content.select("div.content p.assunto-oficio")[0].text
    communication_link = content.select("div.content ul li a")[0].get("href", None)

    url = base_url + communication_link
    
    pdf_response = _get_pdf_content(url)
    pdf_bytes = pdf_response.content
    pdf_file = io.BytesIO(pdf_bytes)

    reader = PdfReader(pdf_file)
    pages = reader.pages
    full_text = ""
    for page in pages:
      full_text = full_text + page.extract_text()

    data.append([published_date, published_title, published_abstract, published_subject, url, full_text])

  return pd.DataFrame(data, columns=["published_date", "published_title", "published_abstract", "published_subject", "url", "full_text"])






In [ ]:
num_processes = 8
pagination = [i for i in range(1, 275)] # There are 274 pages

with multiprocessing.Pool(processes=num_processes) as pool:
    results = list(
        tqdm.tqdm(
        pool.imap(scrape_b3, pagination),
        total=len(pagination)
        )
    )
df = pd.concat(results)
df.to_parquet("./data/b3.parquet", engine="pyarrow")

In [5]:
df = pd.read_parquet("../data/b3.parquet")

In [12]:
df.head()

,published_date,published_title,published_abstract,published_subject,url,full_text
0,17/08/23,145-2023-PRE-Ofício Circular,"Informamos que, em 28/08/2023, entrará em vigo...",Alterações no Manual de Administração de Risco...,https://www.b3.com.br/data/files/83/E1/0B/FA/2...,\n \n \n \nEste documento produz efeitos a pa...
1,17/08/23,075-2023-VNC-Comunicado Externo,"Informamos que, no dia 26/08/2023 (sábado), se...",Sessão de Negociação Simulada – PUMA Trading S...,https://www.b3.com.br/data/files/12/C2/0E/5C/B...,\n \n \n \nEste documento produz efeitos a pa...
2,15/08/23,144-2023-PRE-Ofício Circular,"Em 08/08/2023, por meio de Fato Relevante, a C...","Tratamento de Posições, Garantias e Carteiras ...",https://www.b3.com.br/data/files/F3/01/D4/75/A...,\n \nEste documento produz efeitos a partir d...
3,15/08/23,143-2023-PRE-Ofício Circular,"Informamos que, devido ao fim do horário de ve...",Alteração na Grade Horária de Negociação dos F...,https://www.b3.com.br/data/files/AE/C3/A1/99/E...,\n \nEste documento produz efeitos a partir d...
4,15/08/23,142-2023-PRE-Ofício Circular,"Informamos o lançamento, a partir de 15/08/202...",Lançamento do Índice de Diversidade B3,https://www.b3.com.br/data/files/88/22/63/5D/6...,\n \n \n \nEste documento produz efeitos a pa...
